In [11]:
from fastapi import FastAPI, Form
from pydantic import BaseModel
from loguru import logger
from starlette.responses import HTMLResponse 
from underthesea import word_tokenize
from gensim.utils import simple_preprocess
from keras.layers import Layer
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import sequence
import pickle
import numpy as np
import pandas as pd

In [12]:
app = FastAPI()



def word_processing(sentence):
    sentence = " ".join(simple_preprocess(sentence))
    sentence = [word for word in word_tokenize(sentence.lower(),format='text').split() ]
    return [word for word in sentence if word != ""]

class attention(Layer):
    def __init__(self, return_sequences=True,**kwargs):
        self.return_sequences = return_sequences

        super(attention,self).__init__()
        

    def build(self, input_shape):
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias0", shape=(input_shape[1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias1", shape=(input_shape[1],1)),
        self.b=self.add_weight(name="att_bias2", shape=(input_shape[1],1))

        super(attention,self).build(input_shape)


    def call(self, x):
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        if self.return_sequences:

            return output
        return K.sum(output, axis=1)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "return_sequences": self.return_sequences,
        })
        return config
    
a_file = open('wordtoix_ABSA.pkl','rb')
wordtoix = pickle.load(a_file)

label_aspect = {'RESTAURANT#GENERAL':0,'RESTAURANT#PRICE':1,'RESTAURANT#MISCELLANEOUS':2,'FOOD#QUALITY':3,\
               'FOOD#PRICE':4,'FOOD#STYLE&OPTIONS':5,'DRINK#QUALITY':6,'DRINK#PRICE':7,'DRINK#STYLE&OPTIONS':8,\
                'SERVICE#GENERAL':9,'AMBIENCE#GENERAL':10,'LOCATION#GENERAL':11}
label_aspect_rev =  {v: k for k, v in label_aspect.items()}

label_polarity = {'NONE':0,'NEG':1,'NEU':2,'POS':3}

label_polarity_rev = {v: k for k, v in label_polarity.items()}


@app.get('/') #basic get view
def basic_view():
    return {"WELCOME": "GO TO /docs route, or /post or send post request to /predict "}
@app.get('/predict', response_class=HTMLResponse) #data input by forms
def take_inp():
    return '''<form method="post"> 
    <input type="text" maxlength="100" size = ' 100' name="text" value="type your review"/>  
    <input type="submit"/> 
    </form>'''

@app.post('/predict') #prediction on data
def predict(text:str = Form(...)): #input is from forms
    token_sample = word_processing(text)
    X_predict=[wordtoix[word] for word in token_sample if word in wordtoix]
    X_predict=sequence.pad_sequences([X_predict],382)
    model = load_model('best_model_attetion_absa.hdf5',custom_objects={'attention':attention})
    
    predict= model.predict(X_predict)
    
    i = 0
    label_aspect_pred_lst= []
    for pred in predict[0][0]:
        if pred > 0.0002 :
    #         print(label_aspect_rev[i])
            label_aspect_pred_lst.append(label_aspect_rev[i])
        i = i+1
        
    j =0 
    label_pol_aspect_dict ={}
    for pre_pol in predict[1][0]:
        if pre_pol.argmax() != 0:
    #         print(label_aspect_rev[i],pre_pol.argmax())
            label_pol_aspect_dict[label_aspect_rev[j]]= label_polarity_rev[pre_pol.argmax()]

        j = j +1
    label_aspect_final = {}
    for label in label_pol_aspect_dict.keys():
        if label in label_aspect_pred_lst:
#             print(label,label_pol_aspect_dict[label])
             label_aspect_final[label] = label_pol_aspect_dict[label]
    return {  
            "text": text ,
            "aspect_pol": label_aspect_final,
            }

In [ ]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [40096]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:65144 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:65145 - "GET /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:49324 - "POST /predict HTTP/1.1" 200 OK
